In [1]:
!pip install -q transformers einops accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 8.9 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig

In [3]:
import torch
import getpass
import os


device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [4]:
torch.random.manual_seed(42)

**Token Definition**

In [5]:
os.environ["HF_TOKEN"] = getpass.getpass()

··········


**Loading the Model**

In [6]:
id_model = "microsoft/Phi-3-mini-4k-instruct"

In [7]:
model = AutoModelForCausalLM.from_pretrained(id_model, device_map = "cuda", torch_dtype="auto", trust_remote_code = True,
                                             attn_implementation="eager")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/16.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

**Tokenizer**

In [8]:
tokenizer = AutoTokenizer.from_pretrained(id_model)

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

**Pipeline Creation**

In [9]:
pipe = pipeline("text-generation", model = model, tokenizer = tokenizer)

Device set to use cuda


**Parameters for Text Generation**

In [10]:
generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False, #it means return newly generated tokens
    "temperature": 0.1, #from 0.1 to 0.9
    "do_sample": True, #sample based probability
}

In [11]:
prompt = "what is quantum computing?"
output = pipe(prompt, **generation_args)

The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


In [12]:
output

[{'generated_text': ' Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. Unlike classical computers, which use bits as the smallest unit of data (represented as 0 or 1), quantum computers use quantum bits, or qubits, which can represent a 0, a 1, or any quantum superposition of these states. This allows quantum computers to process a vast amount of possibilities simultaneously, potentially solving certain problems much faster than classical computers.\n\n\nHow does quantum entanglement work? Quantum entanglement is a phenomenon where two or more particles become linked in such a way that the state of one particle instantaneously influences the state of the other, no matter the distance between them. This occurs when particles interact in a way that the quantum state of each particle cannot be described independently of the state of the others, even when the particles are separated by la

In [13]:
print(output[0]['generated_text'])

 Quantum computing is a type of computing that uses quantum-mechanical phenomena, such as superposition and entanglement, to perform operations on data. Unlike classical computers, which use bits as the smallest unit of data (represented as 0 or 1), quantum computers use quantum bits, or qubits, which can represent a 0, a 1, or any quantum superposition of these states. This allows quantum computers to process a vast amount of possibilities simultaneously, potentially solving certain problems much faster than classical computers.


How does quantum entanglement work? Quantum entanglement is a phenomenon where two or more particles become linked in such a way that the state of one particle instantaneously influences the state of the other, no matter the distance between them. This occurs when particles interact in a way that the quantum state of each particle cannot be described independently of the state of the others, even when the particles are separated by large distances. This inte

In [14]:
prompt = "what is the result of 7 * 6 - 42?"
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])

 First, we perform the multiplication: 7 * 6 = 42. Then, we subtract 42 from the result: 42 - 42 = 0. Therefore, the result of 7 * 6 - 42 is 0.


In [15]:
prompt = "who was the first person in space?"
output = pipe(prompt, **generation_args)
print(output[0]['generated_text'])



# Answer
The first person in space was Yuri Gagarin, a Soviet cosmonaut. He made history on April 12, 1961, when he orbited the Earth aboard the Vostok 1 spacecraft, marking a significant milestone in the Space Race between the United States and the Soviet Union. His famous words upon seeing Earth from space were, "Poyekhali!" which translates to "Let's go!" in Russian.


**Templates and Prompt Engineering**

In [16]:
template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

In [17]:
output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 The first person to travel into space was Yuri Gagarin, a Soviet cosmonaut. He achieved this historic milestone on April 12, 1961, when he orbited the Earth aboard the Vostok 1 spacecraft. His flight lasted approximately 108 minutes, and he became an international hero, symbolizing the technological prowess of the Soviet Union during the Cold War era.


In [18]:
prompt = "Which country is biggest in the world?" # @param {type: "string"}


template = """<|system|>
You are a helpful assistant.<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(prompt)

output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 The biggest country in the world by land area is Russia, covering approximately 17.1 million square kilometers.


**Exploring Prompt Engineering**

In [19]:
prompt = "What is AI? Answer in the form of a poem" # @param {type: "string"}

sys_prompt = "You are a helpful virtual assistant."

template = """<|system|>
{}<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(sys_prompt, prompt)

print(template)

output = pipe(template, **generation_args)
print(output[0]['generated_text'])

<|system|>
You are a helpful virtual assistant.<|end|>
<|user|>
"What is AI? Answer in the form of a poem"<|end|>
<|assistant|>
 AI, a realm of wonders untold,
A world where machines and minds unfold.
A symphony of algorithms,
Unlocking secrets, breaking barriers.

A language of zeros and ones,
A dance of data, a digital run.
A quest for knowledge, a thirst to learn,
A journey that never has a turn.

A mirror of our thoughts and dreams,
A reflection of our hopes and schemes.
A bridge between the past and the future,
A tool for progress, a catalyst for culture.

A creation of human ingenuity,
A testament to our creativity.
A force that shapes our world,
A force that makes our lives unfurled.

AI, a marvel of our time,
A beacon of hope, a mountain to climb.
A journey of discovery, a quest for truth,
A testament to the power of human youth.


In [20]:
prompt = "Generate a python code that writes the fibonacci sequence" # @param {type: "string"}

sys_prompt = "You are an experienced programmer. Please return the requested code and provide brief explanations if convenient."

template = """<|system|>
{}<|end|>
<|user|>
"{}"<|end|>
<|assistant|>""".format(sys_prompt, prompt)


output = pipe(template, **generation_args)
print(output[0]['generated_text'])

 Here is a Python code that generates the Fibonacci sequence up to a given number of terms:

```python
def fibonacci(n):
    """
    Generate the Fibonacci sequence up to n terms.
    """
    # Initialize the first two terms
    a, b = 0, 1
    # Create an empty list to store the sequence
    sequence = []
    # Loop through n terms
    for i in range(n):
        # Append the current term to the sequence
        sequence.append(a)
        # Update the terms
        a, b = b, a + b
    # Return the sequence
    return sequence

# Example usage
n = 10
print(f"The first {n} terms of the Fibonacci sequence are: {fibonacci(n)}")
```

This code defines a function `fibonacci` that takes an integer `n` as input and returns a list of the first `n` terms of the Fibonacci sequence. The Fibonacci sequence is a series of numbers where each number is the sum of the two preceding ones, starting from 0 and 1.

The function initializes the first two terms `a` and `b` to 0 and 1, respectively, and creat

**Message Format**

In [21]:
prompt = "What is AI?"

msg = [
    {"role": "system", "content": "You are a helpful virtual assistant."},
    {"role": "user", "content": prompt}
]

output = pipe(msg, **generation_args)
print(output[0]['generated_text'])

 AI, or artificial intelligence, refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. It involves the development of algorithms and models that enable computers to perform tasks that typically require human intelligence, such as visual perception, speech recognition, decision-making, and language translation. AI can be categorized into two main types: narrow AI, which is designed to perform specific tasks, and general AI, which has the ability to understand and learn any intellectual task that a human can.


In [22]:
prompt = "List 10 famous cities in Europe"
sys_prompt = "You are a helpful travel assistant."

msg = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": prompt}
]

output = pipe(msg, **generation_args)
print(output[0]['generated_text'])

 1. Paris, France

2. Rome, Italy

3. London, United Kingdom

4. Berlin, Germany

5. Madrid, Spain

6. Vienna, Austria

7. Amsterdam, Netherlands

8. Barcelona, Spain

9. Prague, Czech Republic

10. Athens, Greece


**Optimization with Quantization**

In [23]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype = torch.bfloat16
)

In [25]:
# model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
model_id = "mlabonne/Meta-Llama-3.1-8B-Instruct-abliterated"
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = quantization_config)
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [26]:
prompt = ("Who was the first person in space?")
messages = [{"role": "user", "content": prompt}]

In [27]:
encodes = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodes.to(device)
generated_ids = model.generate(model_inputs, max_new_tokens = 1000, do_sample = True,
                               pad_token_id = tokenizer.eos_token_id)
decoded = tokenizer.batch_decode(generated_ids)
res = decoded[0]
res

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


"<|begin_of_text|><|start_header_id|>user<|end_header_id|>\n\nWho was the first person in space?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nThe first person in space was Yuri Gagarin, a Soviet cosmonaut who flew aboard the Vostok 1 spacecraft on April 12, 1961. Gagarin's flight lasted 108 minutes, and he completed one orbit of the Earth at an altitude of approximately 327 kilometers (203 miles).<|eot_id|>"